# 📊 Análisis del Presupuesto Público de Chile

> **Objetivo**: Explorar y analizar la distribución del Presupuesto del Sector Público de Chile utilizando datos oficiales de datos.gob.cl

## 📝 Contenido
1. Introducción y Contexto
2. Preparación de Datos
3. Análisis Exploratorio
4. Desarrollo de Visualizaciones
5. Integración con Streamlit

## 🔄 Flujo de Trabajo
Este notebook sirve como prototipo para la aplicación de análisis presupuestario en Streamlit. Exploraremos patrones, desarrollaremos visualizaciones y prepararemos el código para su integración.

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import urllib.request
import json
from typing import Optional, Dict, Any
import streamlit as st  # Para probar funcionalidades de Streamlit

## 🛠️ Configuración y Conexión a la API

Configuramos las variables necesarias para conectarnos a la API de datos.gob.cl y definimos funciones auxiliares para la obtención de datos.

In [2]:
# Configuración de la API
API_URL = 'https://datos.gob.cl/api/3/action/datastore_search'
RESOURCE_ID = '372b0680-d5f0-4d53-bffa-7997cf6e6512'

def fetch_budget_data(url: str, resource_id: str) -> Optional[pd.DataFrame]:
    """
    Obtiene datos del presupuesto desde la API
    
    Args:
        url: URL base de la API
        resource_id: ID del recurso a consultar
        
    Returns:
        DataFrame con los datos o None si hay error
    """
    try:
        full_url = f"{url}?resource_id={resource_id}&limit=10000"
        headers = {'User-Agent': 'Mozilla/5.0', 'Accept': 'application/json'}
        
        print("Consultando API...")
        request = urllib.request.Request(full_url, headers=headers)
        
        with urllib.request.urlopen(request) as response:
            data = json.loads(response.read().decode('utf-8'))
            
            if data.get('success'):
                records = data.get('result', {}).get('records', [])
                if records:
                    df = pd.DataFrame(records)
                    df['Monto Pesos'] = pd.to_numeric(df['Monto Pesos'], errors='coerce')
                    df['Monto Dolar'] = pd.to_numeric(df['Monto Dolar'], errors='coerce')
                    return df.fillna(0)
            
        print("No se obtuvieron datos válidos")
        return None
            
    except Exception as e:
        print(f"Error al obtener datos: {str(e)}")
        return None

## 📥 Carga y Preparación de Datos

Cargamos los datos desde la API y realizamos las transformaciones necesarias.

In [3]:
# Cargar datos
df = fetch_budget_data(API_URL, RESOURCE_ID)

if df is not None:
    print("\nInformación del DataFrame:")
    print("-" * 50)
    print(f"Dimensiones: {df.shape}")
    print(f"\nColumnas disponibles:\n{df.columns.tolist()}")
    print(f"\nTipos de datos:\n{df.dtypes}")
    print(f"\nPrimeras filas:\n{df.head()}")
    
    # Estadísticas básicas
    print("\nEstadísticas de montos:")
    print("-" * 50)
    print(df[['Monto Pesos', 'Monto Dolar']].describe())

Consultando API...

Información del DataFrame:
--------------------------------------------------
Dimensiones: (8924, 10)

Columnas disponibles:
['_id', 'Partida', 'Capitulo', 'Programa', 'Subtitulo', 'Item', 'Asignacion', 'Denominacion', 'Monto Pesos', 'Monto Dolar']

Tipos de datos:
_id               int64
Partida          object
Capitulo         object
Programa         object
Subtitulo        object
Item             object
Asignacion       object
Denominacion     object
Monto Pesos     float64
Monto Dolar       int64
dtype: object

Primeras filas:
   _id Partida Capitulo Programa Subtitulo Item Asignacion  \
0    1      01       01       01        08    0          0   
1    2      01       01       01        08   01          0   
2    3      01       01       01        09    0          0   
3    4      01       01       01        09   01          0   
4    5      01       01       01        10    0          0   

                                        Denominacion  Monto Pesos  Mon

## 📊 Análisis Exploratorio

Desarrollamos visualizaciones exploratorias para entender la distribución y patrones en los datos del presupuesto.

In [4]:
# Funciones de visualización
def plot_top_entities(df: pd.DataFrame, nivel: str, top_n: int = 10) -> go.Figure:
    """Crea visualización de principales entidades"""
    grouped = df.groupby(nivel)['Monto Pesos'].sum().sort_values(ascending=True).tail(top_n)
    
    fig = px.bar(
        grouped.reset_index(),
        x='Monto Pesos',
        y=nivel,
        orientation='h',
        title=f'Top {top_n} {nivel}s por Monto Total',
        labels={'Monto Pesos': 'Monto Total (Pesos)', nivel: nivel.capitalize()},
        text='Monto Pesos',
        color='Monto Pesos',
        color_continuous_scale='viridis'
    )
    
    fig.update_traces(
        texttemplate='$%{text:,.0f}',
        textposition='outside'
    )
    
    fig.update_layout(
        height=400,
        showlegend=False,
        margin=dict(l=10, r=10, t=30, b=10)
    )
    
    return fig

def plot_concentration_analysis(df: pd.DataFrame, nivel: str) -> go.Figure:
    """Crea análisis de concentración"""
    grouped = df.groupby(nivel)['Monto Pesos'].sum().sort_values(ascending=False)
    top_data = grouped.head(10)
    total = grouped.sum()
    
    data_df = pd.DataFrame({
        nivel: top_data.index,
        'Monto': top_data.values,
        'Porcentaje': (top_data / total * 100),
        'Porcentaje Acumulado': (top_data / total * 100).cumsum()
    })
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(
        go.Bar(
            x=data_df[nivel],
            y=data_df['Porcentaje'],
            name='% del Total',
            marker_color='royalblue'
        ),
        secondary_y=False
    )
    
    fig.add_trace(
        go.Scatter(
            x=data_df[nivel],
            y=data_df['Porcentaje Acumulado'],
            name='% Acumulado',
            line=dict(color='firebrick', width=2),
            mode='lines+markers'
        ),
        secondary_y=True
    )
    
    fig.update_layout(
        title=f'Análisis de Concentración - {nivel}',
        height=500,
        xaxis_tickangle=-45,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
    )
    
    return fig

In [5]:
# Generar visualizaciones para cada nivel jerárquico
niveles = ['Partida', 'Capitulo', 'Programa', 'Subtitulo']

for nivel in niveles:
    print(f"\nAnálisis para nivel: {nivel}")
    print("-" * 50)
    
    # Top entidades
    fig_top = plot_top_entities(df, nivel)
    fig_top.show()
    
    # Análisis de concentración
    fig_conc = plot_concentration_analysis(df, nivel)
    fig_conc.show()
    
    # Estadísticas del nivel
    stats = df.groupby(nivel)['Monto Pesos'].agg(['count', 'sum', 'mean', 'std']).round(2)
    print(f"\nEstadísticas para {nivel}:")
    print(stats.head())


Análisis para nivel: Partida
--------------------------------------------------



Estadísticas para Partida:
         count           sum         mean          std
Partida                                               
01          21  5.841233e+07   2781539.67   5105428.84
02          90  4.090111e+08   4544567.83  11535344.00
03          65  1.331446e+09  20483778.48  64021329.54
04          35  2.115480e+08   6044229.06  17030086.54
05        1666  1.340183e+10   8044315.27  41837399.00

Análisis para nivel: Capitulo
--------------------------------------------------



Estadísticas para Capitulo:
          count           sum          mean           std
Capitulo                                                 
01         1974  2.388749e+11  1.210106e+08  2.365128e+09
02          894  6.049679e+10  6.766979e+07  3.226102e+08
03          445  2.925574e+09  6.574324e+06  2.020478e+07
04          455  2.408875e+09  5.294232e+06  2.632993e+07
05          461  7.160703e+09  1.553298e+07  4.841534e+07

Análisis para nivel: Programa
--------------------------------------------------



Estadísticas para Programa:
          count           sum          mean           std
Programa                                                 
01         5959  2.117398e+11  3.553277e+07  1.301383e+09
02         1288  1.935411e+10  1.502648e+07  8.897975e+07
03          391  2.040423e+10  5.218472e+07  2.129151e+08
04          318  1.743188e+10  5.481725e+07  2.382968e+08
05          346  9.026270e+10  2.608749e+08  1.710247e+09

Análisis para nivel: Subtitulo
--------------------------------------------------



Estadísticas para Subtitulo:
           count           sum          mean           std
Subtitulo                                                 
01            58  8.255961e+10  1.423442e+09  1.311227e+10
04            13  4.388208e+09  3.375545e+08  6.444364e+08
05           461  2.623377e+10  5.690623e+07  1.895030e+08
06           136  6.455062e+08  4.746369e+06  2.590509e+07
07           182  1.239089e+09  6.808180e+06  2.204913e+07


## 🚀 Integración con Streamlit

Preparamos las funciones y el código para su integración en la aplicación Streamlit. Las funciones están diseñadas para ser reutilizables y eficientes.

In [6]:
# Ejemplo de código para Streamlit
def create_streamlit_app():
    """
    Ejemplo de cómo se estructuraría la aplicación Streamlit
    """
    print("Estructura de la aplicación Streamlit:")
    print("""
    def run():
        st.title('📊 Análisis del Presupuesto Público de Chile')
        
        # Cargar datos
        df = fetch_budget_data(API_URL, RESOURCE_ID)
        
        if df is not None:
            # Selector de nivel
            nivel = st.selectbox(
                'Seleccione nivel jerárquico:',
                ['Partida', 'Capitulo', 'Programa', 'Subtitulo']
            )
            
            # Visualizaciones
            st.plotly_chart(plot_top_entities(df, nivel))
            st.plotly_chart(plot_concentration_analysis(df, nivel))
            
            # Datos detallados
            with st.expander('Ver datos detallados'):
                st.dataframe(
                    df.groupby(nivel)['Monto Pesos']
                    .agg(['sum', 'mean'])
                    .round(2)
                )
    """)

# Mostrar estructura
create_streamlit_app()

Estructura de la aplicación Streamlit:

    def run():
        st.title('📊 Análisis del Presupuesto Público de Chile')

        # Cargar datos
        df = fetch_budget_data(API_URL, RESOURCE_ID)

        if df is not None:
            # Selector de nivel
            nivel = st.selectbox(
                'Seleccione nivel jerárquico:',
                ['Partida', 'Capitulo', 'Programa', 'Subtitulo']
            )

            # Visualizaciones
            st.plotly_chart(plot_top_entities(df, nivel))
            st.plotly_chart(plot_concentration_analysis(df, nivel))

            # Datos detallados
            with st.expander('Ver datos detallados'):
                st.dataframe(
                    df.groupby(nivel)['Monto Pesos']
                    .agg(['sum', 'mean'])
                    .round(2)
                )
    


## 📝 Conclusiones y Siguientes Pasos

### Conclusiones
- Los datos del presupuesto son accesibles a través de la API de datos.gob.cl
- Las visualizaciones desarrolladas muestran patrones claros de concentración
- La estructura es adecuada para una aplicación Streamlit

### Siguientes Pasos
1. Implementar caché para mejorar rendimiento
2. Agregar más visualizaciones interactivas
3. Incluir análisis temporales
4. Mejorar la documentación
5. Implementar tests automatizados

### Referencias
- [datos.gob.cl](https://datos.gob.cl/)
- [Documentación de Streamlit](https://docs.streamlit.io/)
- [Plotly Documentation](https://plotly.com/python/)